# Análise de texto de fontes desestruturadas e Web

## Aula 12

Nesta aula veremos como realizar **Análise de Tópicos** utilizando bibliotecas de processamento de linguagem natural. Mais precisamente, utilizaremos técnicas de **topic modeling**, um conjunto de algoritmos não supervisionados, ou seja, diferente do que fizemos nas aulas 10 e 11, quando tínhamos as categorias das notícias e treinávamos modelos de Machine Learning para aprender padrões que separam as classes, agora não temos mais estas categorias de antemão. Assim, algoritmos não supervisionados devem inferir categorias ou tópicos que emergem de forma natural nos dados.

A biblioteca utilizada será a **Gensim**.

Para conhecer mais sobre ela, acesse https://radimrehurek.com/gensim/index.html e https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html

## Baixar os dados no Colab

Para baixar os dados no colab, utilize

In [ ]:
!wget https://atd-insper.s3.us-east-2.amazonaws.com/aula12/noticias1.xlsx

## Importando as bibliotecas necessárias

Agora, vamos importar as bibliotecas necessárias:

In [5]:
# para trabalhar com diretórios / sistema operacional
import os

# para trabalhar com expressões regulares
import re

# utilizada para nos indicar o caminho do executável do Python
import sys

# para pandas DataFrame
import pandas as pd

# topic Modeling
import gensim
from gensim import corpora
from gensim import models

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

[nltk_data] Downloading package punkt to /home/gabrielhso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabrielhso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!

Você pode conferir de onde está executando o Python e qual a versão

In [6]:
print('Executável:')
print(sys.executable)

print('\nVersão do Python:')
print(sys.version)

Executável:
/bin/python3

Versão do Python:
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


Vamos conferir em qual diretório iremos trabalhar (é o diretório do notebook)

In [7]:
print('O seu notebook está na pasta:')
print(os.getcwd())

O seu notebook está na pasta:
/home/gabrielhso/Desktop/insper/2024.1/web-scraping/class-repo/aulas/12-lda


Versão das bibliotecas

In [8]:
print('gensim {}'.format(gensim.__version__))
print('nltk {}'.format(nltk.__version__))

gensim 4.3.2
nltk 3.8.1


# Exemplo

Vamos aprender como realizar modelagem de tópicos com o Python. Para isso, vamos criar um DataFrame de exemplo

In [9]:
dfex = pd.DataFrame({'Texto': ['quero ir para Praia curtir o sol. Praia da barra',
                             'não gosto de praia, só de sol',
                             'saudade da sombra e água quente da praia',
                             'tenho um cão que adora comer e passear',
                             'meu cão está doente. Foi algo que ele comeu',
                             'cuide do seu cão para que não fique doente após passear']})
dfex

,Texto
0,quero ir para Praia curtir o sol. Praia da barra
1,"não gosto de praia, só de sol"
2,saudade da sombra e água quente da praia
3,tenho um cão que adora comer e passear
4,meu cão está doente. Foi algo que ele comeu
5,cuide do seu cão para que não fique doente apó...


Vamos considerar que cada linha deste DataFrame representa um **documento**. Ou seja, temos seis documentos nesta base de dados.

Perceba que o DataFrame possui apenas uma coluna de textos, ou seja, não nenhuma variável que indique categorias. Apesar disso, podemos facilmente perceber que alguns documentos falam sobre **praia** e outros que falam sobre **cachorros**.

Para um humano isto é fácil de perceber, agora, veremos como fazer isto no Python!

# Pré-processamento

Utilizaremos diversas técnicas aprendidas durante o curso para limpar e transformar os textos que queremos analisar

Vamos converter o texto para **minúsculo**

In [10]:
dfex['Texto'] = dfex['Texto'].str.lower()
dfex

,Texto
0,quero ir para praia curtir o sol. praia da barra
1,"não gosto de praia, só de sol"
2,saudade da sombra e água quente da praia
3,tenho um cão que adora comer e passear
4,meu cão está doente. foi algo que ele comeu
5,cuide do seu cão para que não fique doente apó...


E separar o texto em uma **lista de palavras** (tokenização)

In [11]:
token_texto = dfex['Texto'].apply(word_tokenize)
print(token_texto)

0    [quero, ir, para, praia, curtir, o, sol, ., pr...
1              [não, gosto, de, praia, ,, só, de, sol]
2    [saudade, da, sombra, e, água, quente, da, praia]
3      [tenho, um, cão, que, adora, comer, e, passear]
4    [meu, cão, está, doente, ., foi, algo, que, el...
5    [cuide, do, seu, cão, para, que, não, fique, d...
Name: Texto, dtype: object


Em seguida, vamos criar um **corpus** textual, que conterá uma representação dos termos ou palavras presentes em cada documento. Aqui, teremos uma representação **Bag of Words**, semelhante ao visto nas duas últimas aulas.

In [12]:
dic = corpora.Dictionary(token_texto)
corpus = [dic.doc2bow(token) for token in token_texto]
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1)],
 [(7, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1)],
 [(3, 2), (7, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)],
 [(15, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)],
 [(0, 1),
  (22, 1),
  (24, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1)],
 [(6, 1),
  (13, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (29, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1)]]

In [13]:
# Conferindo a vetorização de uma frase qualquer
dic.doc2bow(['o', 'quero', 'quero', 'canta', 'para', 'o', 'cão'])

[(5, 2), (6, 1), (8, 2), (22, 1)]

In [14]:
# Entendendo os IDs
dic.token2id

{'.': 0,
 'barra': 1,
 'curtir': 2,
 'da': 3,
 'ir': 4,
 'o': 5,
 'para': 6,
 'praia': 7,
 'quero': 8,
 'sol': 9,
 ',': 10,
 'de': 11,
 'gosto': 12,
 'não': 13,
 'só': 14,
 'e': 15,
 'quente': 16,
 'saudade': 17,
 'sombra': 18,
 'água': 19,
 'adora': 20,
 'comer': 21,
 'cão': 22,
 'passear': 23,
 'que': 24,
 'tenho': 25,
 'um': 26,
 'algo': 27,
 'comeu': 28,
 'doente': 29,
 'ele': 30,
 'está': 31,
 'foi': 32,
 'meu': 33,
 'após': 34,
 'cuide': 35,
 'do': 36,
 'fique': 37,
 'seu': 38}

## Latent Dirichlet allocation (LDA)

Agora, vamos utilizar a biblioteca Gensim para construir um modelo LDA e identificar tópicos no nosso corpus textual.

Considere conferir a documentação do **LdaModel** https://radimrehurek.com/gensim/models/ldamodel.html e seus parâmetros disponíveis.

Primeiramente, vamos considerar dois tópicos e uma palavra

In [15]:
lda = models.ldamodel.LdaModel(corpus, num_topics=2, id2word=dic, passes=25, random_state=1, iterations=100)

lista_topico = lda.print_topics(num_words=1)

for topico in lista_topico:
    print(topico)

(0, '0.077*"cão"')
(1, '0.089*"praia"')


Agora, vamos tentar dois tópicos e mais palavras

In [16]:
lda = models.ldamodel.LdaModel(corpus, num_topics=2, id2word=dic, passes=5, iterations=100)

lista_topico = lda.print_topics(num_words=3)

for topico in lista_topico:
    print(topico)


(0, '0.051*"cão" + 0.050*"doente" + 0.050*"que"')
(1, '0.069*"praia" + 0.053*"da" + 0.038*"que"')


## Plot dos tópicos encontrados

É conveninente ter a possibilidade de visualizar graficamente os tópicos encontrados.

Para isso, utilizaremos uma biblioteca chamada **plotly**.

In [ ]:
lda_top = lda.show_topics(formatted=False)

topico = lda_top[0][1]
topico.sort(key=lambda x: x[1], reverse=True)

palavra = list(zip(*topico))[0]
score = list(zip(*topico))[1]

px.bar(x=palavra, y=score, labels={'x': 'Palavra', 'y': 'Score'}, title='Tópico 0')

In [ ]:
topico = lda_top[1][1]
topico.sort(key=lambda x: x[1], reverse=True)

palavra = list(zip(*topico))[0]
score = list(zip(*topico))[1]

px.bar(x=palavra, y=score, labels={'x': 'Palavra', 'y': 'Score'}, title='Tópico 1')

In [ ]:
lda_top = lda.show_topics(formatted=False)

fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=('Tópico 1', 'Tópico 2'))

for i in range(2):
    topico = lda_top[i][1]
    topico.sort(key=lambda x: x[1], reverse=True)

    palavra = list(zip(*topico))[0]
    score = list(zip(*topico))[1]
    
    fig.add_trace(go.Bar(x=palavra, y=score,
                         marker=dict(color=score, coloraxis="coloraxis")), 1, i+1)
    
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
fig.show()

Verificando se é **stopword**

In [ ]:
'que' in stopwords.words('portuguese')

Com quais tópicos nossas frases originais mais se assemelham?

In [ ]:
lda_res = lda[corpus]

for doc, as_text in zip(lda_res, dfex['Texto']):
    print(doc, as_text)

### Testando para novas frases

In [ ]:
doc = ['meu cão sabe correr',
       'sol quente e praia, quero demais']

token_frase = [word_tokenize(frase) for frase in doc]

corpus_ex = [dic.doc2bow(lista) for lista in token_frase]
lda_res = lda[corpus_ex]

for doc, as_text in zip(lda_res, doc):
    print(doc, as_text)

# Exercícios

**Exercício 1** Crie uma função de pré-processamento de textos. A função recebe uma string e:
- Remove caracteres numéricos
- Remove acentuação
- Transforma tudo para minúsculo
- Remove stopwords
- Retorna a `lista de palavras` contidas na string

In [ ]:
def preproc_texto(txt):
    # Faça aqui
    return #Adicione aqui a lista a ser retornada

# Resultado esperado ['hey', 'vamos', 'aprender', 'nlp', 'estudo', 'vezes', 'semana']
preproc_texto('Hey, vamos aprender NLP??? Eu estudo 3 vezes na semana.')

**Exercício 2** Aplique LDA à base `noticias_1.xlsx` para identificar os tópicos que emergem a partir da análise do **`Título`** das notícias. Você mesmo pode atribuir um número de tópicos e quantidade de palavras na análise (teste diferentes valores).

OBS: Faça todo o pré-processamento, tokenização, aplicação do LDA, etc. conforme visto no handout.

**Exercício 3** Repita o exercício anterior com outras bases. Você pode utilizar os arquivos anexos no Blackboard ou fazer o download utilizando o notebook de scraping da IstoÉ (aulas anteriores).

Teste diferentes quantidades de tópicos e palavras na análise.

**Exercício 4** Altere a função de limpeza. Adicione palavras para serem desconsideradas, além das contidas na lista de stop words utilizada. Você consegue perceber alterações nos resultados?

**Exercício 5** Plote um gráfico de barras para cada tópico. Cada gráfico deve conter as palavras (e scores) pertencentes ao tópicos identificados.

**Exercício 6** Adicione um Stemmer à sua etapa de pré-processamento. Aplique aos dados de notícia e avalie os resultados.

https://pt.wikipedia.org/wiki/Stemização

https://www.kite.com/python/docs/nltk.SnowballStemmer

https://snowballstem.org/algorithms/portuguese/stemmer.html

Dicas:

Cria o stemmer
```python
stemmer = SnowballStemmer("portuguese")
```

Aplica em uma lista de palavras

```python
lista_stem = [stemmer.stem(word) for word in lista_palavras]
```